# EEG Biometrics


In [ ]:
# Run this cell to load required libraries and mount your Drive folder

import numpy as np
from matplotlib import pyplot as plt
from google.colab import drive
import os
import json
import csv


drive.mount('/content/drive')


In [ ]:
# Run this cell to navigate through the Drive filesystem and retrieve the dataset

dataset_path = '/content/drive/MyDrive/ml2-eeg-biometrics/raw-data/'

task_duration = 5 # Task average duration is 9 seconds

dataset = {}

left_hand_task = 769
right_hand_task = 770
both_feet_task = 771
idle_task = 780

folder_list = os.listdir(dataset_path)

# Navigate directories and files in the dataset
for participant_folder in folder_list:
  #print("Participant " + participant_folder)
  dataset[participant_folder] = {}
  for folder in os.listdir(dataset_path + participant_folder):
    #print("Folder: " + folder)
    date = folder.split("_")[1]
    dataset[participant_folder][date] = {}
    identifier = 1
    for trial in os.listdir(dataset_path + participant_folder + "/" + folder):
      key = "trial_" + str(identifier)
      dataset[participant_folder][date][key] = []
      identifier += 1
      
      npz_data = np.load(dataset_path + participant_folder + "/" + folder+ "/" + trial)
      npz_data_dict = dict(npz_data)
      sample_rate = npz_data_dict['SampleRate'][0]
      marks = npz_data_dict['MarkOnSignal'] 
      data = npz_data_dict['signal']
      for mark in marks:
        if mark[1] == right_hand_task: 
          start_index = mark[0]
          dataset[participant_folder][date][key].append(data[start_index])

  print("Participant " + participant_folder + " done!") 

In [ ]:
# Dataset reader
for participant in dataset:
  print(participant)
  for date in dataset[participant]:
    print(date)
    for trial in dataset[participant][date]:
      print(trial + ":")
      print(dataset[participant][date][trial])
    print("\n")
  print("\n")

In [ ]:
example_file_name = '/content/drive/MyDrive/ml2-eeg-biometrics/raw-data/A1/A1_20190613/NSsignal_2019_06_13_19_16_33.npz'

npz_data = np.load(example_file_name)
npz_data_dict = dict(npz_data)
sample_rate = npz_data_dict['SampleRate'][0]
marks = npz_data_dict['MarkOnSignal'] 
data = npz_data_dict['signal']
channel_labels = np.array(['F3', 'F1', 'Fz', 'F2', 'F4', 'FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6'])

with open('eeg_dataset_example.csv', "w") as csv_file:
        trial = 0
        csv_file.write("Participant A1")
        csv_file.write("\n")
        csv_file.write("Day 20190613")
        csv_file.write("\n")
        csv_file.write("Run 2019_06_13_19_16_33")        
        csv_file.write("\n")
        for mark in marks:
          if mark[1] == 770: 
            start_index = mark[0]
            #dataset["A1"]["20190613"]["trial_1"] = data[start_index]
            csv_file.write("Trial" + str(trial))
            csv_file.write("\n")
            for channel in channel_labels:
              csv_file.write(str(channel) + ",")
            csv_file.write("\n")
            for i in range(0, channel_labels.size):
              csv_file.write(str(data[start_index][i]) + ",")
            csv_file.write("\n")
            trial += 1
print("Done writing CSV file")

In [ ]:
# Run this cell to save the changes

drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')